In [ ]:
'''
Issues
- ISM
    - SAM segments low quality masks with specific objects (Found a lot of mask proposal, but low quality)
    - Requires CUDA Memory (Around 8-10GB VRAM)
- PEM
    - Occlusion (pointclouds should be visible as much as possible)
    - Filled Masks (masks should not be filled, holes should be holes)
    - CUDA Memory (increase in the number of masks resulted in the greater need of CUDA memory)
      [ one or two masks consume only 3-4GB VRAM ]
'''

In [ ]:
# Make sure you have Anaconda Installed
# export PATH=~/anaconda3/bin:$PATH

# For 6d runpod
# export PATH=/workspace/Downloads/Anaconda/bin:$PATH

conda init
conda activate sam6d

cd SAM-6D/SAM-6D
sh prepare.sh

# - Template, ISM, PEM, split them into 3 shell files

In [ ]:
# Install blender-3.1.1-linux-x64 (It should be a folder)
# or
# Install blender-3.1.1-linux-x64.tar.xz
# and tar.xz blender-3.1.1-linux-x64.tar.xz (Uncompress)

cd Render

export CAD_PATH=/path_to/SAM-6D/SAM-6D/Data/Example/obj_000005.ply    # path to a given cad model(mm)
export RGB_PATH=/path_to/SAM-6D/SAM-6D/Data/Example/rgb.png           # path to a given RGB image
export DEPTH_PATH=/path_to/SAM-6D/SAM-6D/Data/Example/depth.png       # path to a given depth map(mm)
export CAMERA_PATH=/path_to/SAM-6D/SAM-6D/Data/Example/camera.json    # path to given camera intrinsics
export OUTPUT_DIR=/path_to/SAM-6D/SAM-6D/Data/Example/outputs         # path to a pre-defined file for saving results

blenderproc run --custom-blender-path /path_to/blender-3.1.1-linux-x64 render_custom_templates.py --output_dir $OUTPUT_DIR --cad_path $CAD_PATH --colorize True  

# If there is an error with: error while loading shared libraries: libXi.so.6: cannot open shared object file: No such file or directory
apt-get update
apt-get install libxi6 libxrender1 libxrandr2 libxcursor1 libxinerama1 libgl1-mesa-glx libglib2.0-0 libsm6 libice6

# If there is an error with: ImportError: ('Unable to load EGL library', 'EGL: cannot open shared object file: No such file or directory', 'EGL', None)
apt-get update
apt-get install libegl1-mesa libgles2-mesa

# And try running blenderproc run....... again
# The default settings from Github would generate 42 templates in total
# If the templates don't show up, its likely that it cannot detect your GPU


# apt-get all, in case you want to ignore all the error at first
apt-get update
apt-get install libxi6 libxrender1 libxrandr2 libxcursor1 libxinerama1 libgl1-mesa-glx libglib2.0-0 libsm6 libice6 libegl1-mesa libgles2-mesa

In [ ]:
cd ../Instance_Segmentation_Model
export SEGMENTOR_MODEL=sam

python run_inference_custom.py --segmentor_model $SEGMENTOR_MODEL --output_dir $OUTPUT_DIR --cad_path $CAD_PATH --rgb_path $RGB_PATH --depth_path $DEPTH_PATH --cam_path $CAMERA_PATH

# If the 'Numpy version error' pops up, you might have to downgrade or install a specific version of numpy to solve it
pip install numpy==1.24.1

# And try running the command python run_inference_custom.py ............ again

In [ ]:
cd ../Pose_Estimation_Model
export SEG_PATH=$OUTPUT_DIR/sam6d_results/detection_ism.json

python run_inference_custom.py --output_dir $OUTPUT_DIR --cad_path $CAD_PATH --rgb_path $RGB_PATH --depth_path $DEPTH_PATH --cam_path $CAMERA_PATH --seg_path $SEG_PATH

# The error with pointnet2, from pkg_resources import packaging, might pops up
# It could be solve by installing setup tools that mathces the python in conda
conda install anaconda::setuptools

# If there is more problem with pointnet2, you might have to do this (Local installation)
sudo apt-get install gcc-9 g++-9
export CC=gcc-9
cd /path_to/SAM-6D/SAM-6D/Pose_Estimation_Model/model/pointnet2

python setup.py clean
python setup.py install

# And try running the command python run_inference_custom.py ............ again

# In this part here, there might be an error that states the insufficient CUDA memory
# in which, currently, i found a way to solve it by reducing the n-proposal down (Pose_Estimation_Model/config/base.yaml)
# but the result might be poor due to the smaller number of proposals used to compute the matching scores.

In [ ]:
# With our camera, in the camera.json, the depth scale should be set to 0.001
{"cam_K": [607.060302734375, 0.0, 639.758056640625, 0.0, 607.1031494140625, 363.29052734375, 0.0, 0.0, 1.0], "depth_scale": 0.001}